<a href="https://colab.research.google.com/github/harshatejas/pytorch_custom_object_detection/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/harshatejas/pytorch_custom_object_detection.git

Cloning into 'pytorch_custom_object_detection'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (417/417), done.
remote: Compressing objects: 100% (414/414), done.
remote: Total 417 (delta 15), reused 375 (delta 3), pack-reused 0
Receiving objects: 100% (417/417), 36.42 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
%cd /content/pytorch_custom_object_detection

/content/pytorch_custom_object_detection


In [3]:
# Imports
import os
import numpy as np
import pandas as pd
import cv2

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [4]:
# Hyperparameters
test_set_length = 40 		 # Test set (number of images)
train_batch_size = 2  		 # Train batch size
test_batch_size = 1    		 # Test batch size
num_classes = 6        		 # Number of classes
learning_rate = 0.005  		 # Learning rate
num_epochs = 40      	     # Number of epochs
output_dir = "saved_model"   # Output directory to save the model

In [5]:
# Helper functions 
def create_label_txt(path_to_csv):

	data = pd.read_csv(path_to_csv)
	labels = data['class'].unique()

	labels_dict = {}

	# Creat dictionary from array
	for index, label in enumerate(labels):
		labels_dict.__setitem__(index, label)

	# We need to create labels.txt and write labels dictionary into it
	with open('labels.txt', 'w') as f:
		f.write(str(labels_dict))

	return labels_dict	

def parse_one_annot(path, filename, labels_dict):

	data = pd.read_csv(path)

	class_names = data['class'].unique()
	classes_df = data[data["filename"] == filename]["class"]
	classes_array = classes_df.to_numpy()
	
	boxes_df = data[data["filename"] == filename][["xmin", "ymin", "xmax", "ymax"]]
	boxes_array = boxes_df.to_numpy()
	
	classes = []
	for key, value in labels_dict.items():
		for i in classes_array:
			if i == value:
				classes.append(key)

	# Convert list to tuple
	classes = tuple(classes)

	return boxes_array, classes

def get_model(num_classes):

	# Load an pre-trained object detectin model (in this case faster-rcnn)
	model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

	# Number of input features
	in_features = model.roi_heads.box_predictor.cls_score.in_features

	# Replace the pre-trained head with a new head
	model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

	return model


def get_transforms(train):

	transforms = []

	# Convert numpy image to PyTorch Tensor
	transforms.append(T.ToTensor())

	if train:
		# Data augmentation
		transforms.append(T.RandomHorizontalFlip(0.5))

	return T.Compose(transforms) 

In [6]:
class CardsDataset(torch.utils.data.Dataset):

	""" The dataset contains images of playing cards 
		The dataset includes images of king, queen, jack, ten, nine and ace playing cards"""

	def __init__(self, dataset_dir, csv_file, labels_dict, transforms = None):

		self.dataset_dir = dataset_dir
		self.csv_file = csv_file
		self.transforms = transforms
		self.labels_dict = labels_dict
		self.image_names = [file for file in sorted(os.listdir(os.path.join(dataset_dir))) if file.endswith('.jpg') or file.endswith('.JPG')]

	def __getitem__(self, index):

		image_path = os.path.join(self.dataset_dir, self.image_names[index])
		image = cv2.imread(image_path)
		# Convert BGR to RGB
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

		box_array, classes = parse_one_annot(self.csv_file, self.image_names[index], self.labels_dict)
		boxes = torch.as_tensor(box_array, dtype = torch.float32)

		labels = torch.tensor(classes, dtype=torch.int64)
		
		image_id = torch.tensor([index])
		area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

		iscrowd = torch.tensor(classes, dtype=torch.int64)
		target = {}
		target["boxes"] = boxes
		target["labels"] = labels
		target["image_id"] = image_id
		target["area"] = area
		target["iscrowd"] = iscrowd

		if self.transforms is not None:
			image, target = self.transforms(image, target)

		return image, target

	def __len__(self):

		return len(self.image_names)

In [7]:
# Setting up the device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

labels_dict = create_label_txt("cards_dataset/train_labels.csv")

# Define train and test dataset
dataset = CardsDataset(dataset_dir = "cards_dataset/train/", csv_file = "cards_dataset/train_labels.csv",
                        labels_dict = labels_dict, transforms = get_transforms(train = True))

dataset_test = CardsDataset(dataset_dir = "cards_dataset/train/", csv_file = "cards_dataset/train_labels.csv", 
                        labels_dict = labels_dict, transforms = get_transforms(train = False))

# Split the dataset into train and test
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-test_set_length])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-test_set_length:])

# Define train and test dataloaders
data_loader = torch.utils.data.DataLoader(dataset, batch_size = train_batch_size, shuffle = True,
                num_workers = 2, collate_fn = utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size = test_batch_size, shuffle = False,
                num_workers = 2, collate_fn = utils.collate_fn)

print(f"We have: {len(indices)} images in the dataset, {len(dataset)} are training images and {len(dataset_test)} are test images")

We have: 358 images in the dataset, 318 are training images and 40 are test images


In [8]:
# Get the model using helper function
model = get_model(num_classes)
model.to(device = device)

# Construct the optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = learning_rate, momentum = 0.9, weight_decay = 0.0005)

# Learning rate scheduler decreases the learning rate by 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

for epoch in range(num_epochs):
    
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq = 10)
    lr_scheduler.step()
    # Evaluate on the test dataset
    evaluate(model, data_loader_test, device = device)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Save the model state	
torch.save(model.state_dict(), output_dir + "/model")

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth



Epoch: [0]  [  0/159]  eta: 0:03:05  lr: 0.000037  loss: 2.0135 (2.0135)  loss_classifier: 1.6735 (1.6735)  loss_box_reg: 0.3385 (0.3385)  loss_objectness: 0.0004 (0.0004)  loss_rpn_box_reg: 0.0011 (0.0011)  time: 1.1664  data: 0.2763  max mem: 3392
Epoch: [0]  [ 10/159]  eta: 0:01:04  lr: 0.000353  loss: 1.6322 (1.4553)  loss_classifier: 1.4057 (1.1858)  loss_box_reg: 0.2458 (0.2622)  loss_objectness: 0.0020 (0.0035)  loss_rpn_box_reg: 0.0036 (0.0038)  time: 0.4361  data: 0.0332  max mem: 3659
Epoch: [0]  [ 20/159]  eta: 0:00:54  lr: 0.000669  loss: 0.7879 (1.0224)  loss_classifier: 0.4115 (0.7685)  loss_box_reg: 0.1901 (0.2444)  loss_objectness: 0.0039 (0.0063)  loss_rpn_box_reg: 0.0026 (0.0032)  time: 0.3534  data: 0.0095  max mem: 3659
Epoch: [0]  [ 30/159]  eta: 0:00:50  lr: 0.000985  loss: 0.4299 (0.8575)  loss_classifier: 0.2581 (0.6045)  loss_box_reg: 0.1901 (0.2433)  loss_objectness: 0.0069 (0.0068)  loss_rpn_box_reg: 0.0018 (0.0029)  time: 0.3683  data: 0.0103  max mem: 3659

In [9]:
from google.colab import files
files.download(output_dir + "/model")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>